In [55]:
f = open("input.txt", "r")
input = list(f.read().split("\n"))

In [69]:
def in_range(x, r):
    return x >= r[1] and x < r[1] + r[2]

def map_range(x, r):
    return r[0] + (x - r[1])

def parse_row(row):
    return tuple(map(int, filter(lambda x: len(x) > 0, row.split(" "))))

seed_ids = parse_row(input[0][6:])

split_inputs = []
input_copy = input[2:]

while "" in input_copy:
    index = input_copy.index("")
    split_inputs.append(list(map(parse_row, input_copy[1:index])))
    input_copy = input_copy[index + 1:]
split_inputs.append(list(map(parse_row, input_copy[1:])))

def map_seed(seed_id):
    for ranges in split_inputs:
        for range_group in ranges:
            if in_range(seed_id, range_group):
                seed_id = map_range(seed_id, range_group)
                break

    return seed_id


print(f"Solution: {min(list(map(map_seed, seed_ids))):,}")


Solution: 51,580,674


In [67]:
def do_range_overlap(input_range, test_range):
    min_input, max_input = input_range[0], input_range[1]
    min_test, max_test = test_range[1], test_range[1] + test_range[2] - 1

    return min_input <= max_test and max_input >= min_test

def find_range_overlap(input_range, test_range):
    if not do_range_overlap(input_range, test_range):
        return [input_range], None
    
    min_input, max_input = input_range[0], input_range[1]
    min_test, max_test = test_range[1], test_range[1] + test_range[2] - 1

    overlap = max(min_input, min_test), min(max_input, max_test)

    remaining = []
    if min_input < overlap[0]:
        remaining.append((min_input, overlap[0] - 1))
    if max_input > overlap[1]:
        remaining.append((overlap[1] + 1, max_input))

    return remaining, (overlap[0] - min_test + test_range[0], overlap[1] - min_test + test_range[0])

def map_ranges(input_ranges, test_ranges, output_ranges):
    if len(test_ranges) == 0:
        return input_ranges + output_ranges
    
    test_range, test_ranges = test_ranges[0], test_ranges[1:]
    new_input_ranges, new_output_ranges = [], []
    for input_range in input_ranges:
        new_input_range, new_output_range = find_range_overlap(input_range, test_range)
        new_input_ranges += new_input_range
        if not new_output_range is None:
            new_output_ranges.append(new_output_range)

    return map_ranges(new_input_ranges, test_ranges, output_ranges + new_output_ranges)

def GetSolution():
    starting_ranges = []
    for i in range(0, len(seed_ids), 2):
        starting_ranges.append((seed_ids[i], seed_ids[i] + seed_ids[i+1] - 1))

    for split_input in split_inputs:
        starting_ranges = map_ranges(starting_ranges, split_input, [])

    return min([min_index for min_index, _ in starting_ranges])
print(f"Solution: {GetSolution():,}")

Solution: 99,751,240


In [66]:
%timeit GetSolution()

1.26 ms ± 17.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
